In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn 
from torch.nn import functional as F
from torchviz import make_dot
from tensorboardX import SummaryWriter
from spair.models import Spair
from datetime import datetime
import numpy as np
from spair.metric import mAP

In [1]:

img = torch.rand((32,1,128,128))
device = torch.device("cpu")
dt = datetime.today().strftime('%b-%d')
writer = SummaryWriter('logs_v2/%s' % dt)

spair = Spair((1, 128,128),writer,device,)

spair_params = dict(spair.named_parameters())
decoder_in_weights = spair_params['object_decoder.dense0.weight']
encoder_out_param = spair_params['object_encoder.out.weight']
print('making dot')
# make_dot(spair(img))


# make_dot(spair(img), params=dict(spair.named_parameters()))

NameError: name 'torch' is not defined

In [92]:

class test_grad_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.output = nn.Linear(4, 4)
        self.output.weight.data.fill_(1)
        self.output.bias.data.fill_(0)
    def forward(self, x):
        return self.output(x)
        
net = test_grad_net()

x = torch.ones(2, 8, dtype=torch.float, requires_grad=False) * 0.1# x1 and x2 batch, batch size = 2
y = torch.tensor([[1,.1,.1,.1,.1,.1,1,.1],[.1,1,.1,.1,.1,.1,.1,1]], dtype=torch.float, requires_grad=False)

x1 = x[:, :4]
x2 = x[:, 4:] * 2
y1 = y[:, :4]
y2 = y[:, 4:]
out1 = net(x1)
out2 = net(x2)

out = torch.cat([out1, out2], dim=-1)

loss = F.binary_cross_entropy(out, y, reduction='sum')

loss.backward()

W = net.output.weight
W.grad


tensor([[1.6250, 1.6250, 1.6250, 1.6250],
        [1.6250, 1.6250, 1.6250, 1.6250],
        [0.8750, 0.8750, 0.8750, 0.8750],
        [0.8750, 0.8750, 0.8750, 0.8750]])

# Testing mean Average Precision metirc


In [19]:
batch_size = 1
grid_size = 11
z_where = np.zeros((batch_size,grid_size * grid_size,4))

z_where[...,2:] = 0.5 

z_pres = np.ones((batch_size,grid_size * grid_size,1))

ground_truth_bbox = np.zeros((batch_size, 20, 4))
ground_truth_bbox[...,2:] = 0.5
truth_bbox_digit_count = np.ones((batch_size, 1)) * 20


# conversion stuff
z_where_t = torch.tensor(z_where, dtype=torch.float32)
z_where_t = z_where_t.view(batch_size,grid_size, grid_size, -1).permute(0,3,1,2)
z_pres_t = torch.tensor(z_pres, dtype=torch.float32)
z_pres_t = z_pres_t.view(batch_size,grid_size, grid_size, -1).permute(0,3,1,2)
ground_truth_bbox_t = torch.tensor(ground_truth_bbox, dtype=torch.float32)
truth_bbox_digit_count_t = torch.tensor(truth_bbox_digit_count, dtype=torch.float32)

result = mAP(z_where_t, z_pres_t, ground_truth_bbox_t, truth_bbox_digit_count_t)
print(result.item())
